## <u>PBF-Prep</u>  
It's necessary to download the State of interest.  
⭐ Download the administrative area/region from https://download.geofabrik.de/  
(It's also possible to get the PBF extract through [BBBike](https://download.bbbike.org/osm/), data is updated weekly instead of daily [More info](https://wiki.openstreetmap.org/wiki/Planet.osm#Extracts))  
If it spans multiple states, use `osmium merge` to merge the two states.  
⭐ Needed python packages are:   
<b>├── geopandas  
  └── folium  </b>
⭐ Then run the cell below to select a bounding box for the area on the map. (Uses the last made rectangle)  
The bbox is fed into an osmium command to pare down the pbf to the area of interest.  

In [ ]:
from ipyleaflet import Map, Marker, GeomanDrawControl
import geopandas as gpd
from shapely.geometry import shape
import os

m = Map(center=(15,10), zoom=2.5, scroll_wheel_zoom=True, 
        layout={"height": "800px", "width": "100%"}
        )
draw_control = GeomanDrawControl(edit=False, drag=False, cut=False, rotate=False, circlemarker={}, polyline={}, polygon={})
draw_control.rectangle = {"pathOptions": {"color": "black","fillOpacity": 0.5}}
def draw_update(target, action, geo_json):
    global west, south, east, north
    if action == "create":
        west, south, east, north = gpd.GeoSeries([shape(geo_json[0])])[0].bounds
    pass
draw_control.on_draw(draw_update)
m.add(draw_control)

Map(center=[15, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

### ⭐ Adjust filenames and run cell  
I know making os.system calls is not ideal. Working on moving this to pyosmium so I can also drop the requirement for the cli tool. 

In [ ]:
startingfile = "./pbf/place-date.osm.pbf"
outputfile = "./pbf/area-of-interest" #Don't include "osm.pbf"

print(f"{west},{south},{east},{north}")
os.system(f"osmium extract -b {west},{south},{east},{north} {startingfile} -Oo {outputfile}.osm.pbf")
#              -b = bounding box | lat/lng restrictions | file to act on | -o = output | output file
os.system(f"osmium tags-filter {outputfile}.osm.pbf w/highway -Oo {outputfile}-roads.osm.pbf")
#                                                   filter for highways
os.system(f"osmium tags-filter {outputfile}.osm.pbf man_made=surveillance -Oo {outputfile}-surveillance.osm.pbf")
#                                                   filter for surveillance

0

## If the above cell worked, ignore the stuff below
If the osmium CLI is not set up properly/not working. Then feed in an extract from BBBike to the function below to generate the needed files.  


In [ ]:
startingfile = "./pbf/bbbike-area-of-interest.osm.pbf"
outputfile = "./pbf/area-of-interest" #Don't include "osm.pbf"

# Files were similar in size as the command line output
# surveillance file only differed by 16 bytes 
# roads by 600KB out of 39MB

import osmium as omi
from osmium import FileProcessor as FProc

with omi.BackReferenceWriter(outputfile+"-roads"+".osm.pbf", ref_src=startingfile, overwrite=True) as writer:
    hwfilter = omi.filter.KeyFilter("highway")
    roads = FProc(startingfile, omi.osm.WAY)\
            .with_filter(hwfilter)
    for obj in roads:
        writer.add(obj)

with omi.BackReferenceWriter(outputfile+"-surveillance"+".osm.pbf", ref_src=startingfile, overwrite=True) as writer:
    manmfilter = omi.filter.KeyFilter("man_made")
    surveillance = FProc(startingfile)\
            .with_filter(manmfilter)
    for obj in surveillance:
        if obj.tags["man_made"] == "surveillance":
             writer.add(obj)  